**Removing December from financial data from 2023**

In [ ]:
begin transaction 
delete from CaptiolOne2023
where month(Posted_Date) = '12'
commit

begin transaction 
delete from Chase2023
where month(Posting_Date) = '12'
commit

**Finances 2022**

In [ ]:
with cte as (
select 
month(Posting_Date) as Month,
sum(
case when Amount > 0 then cast(Amount as int) else 0
end) as Earnings,
sum(
case when Amount < 0 then abs(cast(Amount as int)) else 0
end) as Expenses
from Chase2022
group by month(Posting_Date)
), cte_2 as (
select 
month(Posted_Date) as Month,
cast(sum(Debit) as int) as Expenses
from CapitolOne2022
group by month(Posted_Date)
)
select 
a.Month,
Earnings,
a.Expenses + b.Expenses as Expenses
into finances_2022
from cte a join cte_2 b
on a.Month = b.Month

In [1]:
select *
from finances_2022

(12 rows affected)

Total execution time: 00:00:00.114

Month,Earnings,Expenses
1,1395,840
2,1469,1021
3,1258,887
4,1547,1118
5,1653,2366
6,4049,1660
7,3029,1711
8,2002,1999
9,2189,1369
10,2475,1835


**Finances 2023**

In [ ]:
with cte_1 as (
select 
month(Posting_Date) as Month,
sum(case
	when Amount > 0 then cast(Amount as int) else 0
end) Earnings,
sum(case
	when Amount < 0 then abs(cast(Amount as int)) else 0
end) as Expenses
from Chase2023 
group by month(Posting_Date)
union
select
month(Posting_Date) as Month,
sum(case
	when Amount > 0 then cast(Amount as int) else 0
end) Earnings,
sum(case
	when Amount < 0 then abs(cast(Amount as int)) else 0
end) as Expenses
from Chase_12
group by month(Posting_Date)
), cte_2 as (
select 
month(Posted_Date) as Month,
sum(cast(Debit as int)) as Expenses
from CaptiolOne2023
group by month(Posted_Date)
union 
select 
month(Posted_Date) as Month,
sum(cast(Debit as int)) as Expenses
from Capitol_One_12
group by month(Posted_Date)
)
select 
a.Month,
Earnings,
a.Expenses + b.Expenses as Expenses
into finances_2023
from cte_1 a join cte_2 b
on a.Month = b.Month

In [2]:
select *
from finances_2023

(12 rows affected)

Total execution time: 00:00:00.048

Month,Earnings,Expenses
1,4039,3605
2,2098,1830
3,1900,1173
4,2219,2428
5,1955,2773
6,3819,3845
7,2051,2300
8,2068,3106
9,5010,3122
10,2707,3205


**Total Finances 2022 & 2023**

In [ ]:
select '2022' as 'Year',
sum(Earnings) as 'Total Earnings',
sum(Expenses) as 'Total Expenses'
into total_finances
from finances_2022
union
select '2023' as 'Year',
sum(Earnings) as 'Total Earnings',
sum(Expenses) as 'Total Expenses'
from finances_2023

In [11]:
select *
from total_finances

(2 rows affected)

Total execution time: 00:00:00.019

Year,Total Earnings,Total Expenses
2022,26649,19050
2023,32609,35121


**All Finances**

In [15]:
select 
datename(month, dateadd(month, Month -1, '19000101')) as Month,
'2022' as 'Year',
Earnings,
Expenses
into finances
from finances_2022
union all 
select 
datename(month, dateadd(month, Month -1, '19000101')) as Month,
'2023' as 'Year',
Earnings,
Expenses
from finances_2023
union all
select 
'Total',
Year,
[Total Earnings],
[Total Expenses]
from total_finances

(26 rows affected)

Total execution time: 00:00:00.022

In [16]:
select *
from finances

(26 rows affected)

Total execution time: 00:00:00.014

Month,Year,Earnings,Expenses
January,2022,1395,840
February,2022,1469,1021
March,2022,1258,887
April,2022,1547,1118
May,2022,1653,2366
June,2022,4049,1660
July,2022,3029,1711
August,2022,2002,1999
September,2022,2189,1369
October,2022,2475,1835
